In [1]:
import pandas as pd
import numpy as np
import os
from os.path import isfile, join
import xml.etree.ElementTree as ET
import copy
import matplotlib.pyplot as plt
from matplotlib import rcParams
import sys

pd.options.display.float_format = '{:.2f}'.format

rcParams["errorbar.capsize"] = 8.
#rcParams['font.family'] = 'Times New Roman'
rcParams['font.size'] = '14'
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

sys.path.append('..')
from models.OnlineFleet import  from_xml

In [2]:
instances = ['large']
data_alpha = []
data_beta = []
columns = ['Instance', 'Fitness', 'Distance', 'J1', 'J2', 'J3', 'J4', 'Fleet size', 'Execution time (s)']

In [4]:
for instance in instances:
    main_folder = f'../data/instances/{instance}/'
    instances_folders = [os.path.join(main_folder, i) for i in os.listdir(main_folder) if (os.path.isdir(os.path.join(main_folder, i)) and i!='old')]
    for folder in instances_folders:
        instance_name = folder.split('/')[-1]
        opt_folders = [i for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))]
        num = max([i.split('_')[-1] for i in opt_folders]) 
        opt_folder = f'{folder}/opt_{num}/'

        num = max([int(i.split('_')[-1]) for i in os.listdir(opt_folder)])

        # [ALPHA GA]
        folder = f'{opt_folder}alphaGA_fleetsize_{num}/'
        fitness_df = pd.read_csv(folder+'optimization_iterations.csv')
        hyperparms_df = pd.read_csv(folder+'hyper-parameters.csv')
        costs_df = pd.read_csv(folder+'costs.csv')

        exec_time = float(hyperparms_df.loc[hyperparms_df.iloc[:, 0] == 'Algorithm time'].iloc[0, 1])
        fleet_size = hyperparms_df.loc[hyperparms_df.iloc[:, 0] == 'm'].iloc[0, 1]

        fitness = fitness_df.iloc[-1, 1]
        J1, w1 = costs_df.iloc[1,1], costs_df.iloc[0,1]
        J2, w2 = costs_df.iloc[1,2], costs_df.iloc[0,2]
        J3, w3 = costs_df.iloc[1,3], costs_df.iloc[0,3] 
        J4, w4 = costs_df.iloc[1,4], costs_df.iloc[0,4]
        J5, w5 = costs_df.iloc[1,5], costs_df.iloc[0,5]

        fleet = from_xml(folder+'result_instance.xml', True)
        fleet.iterate()
        feasible, distance, accept = fleet.feasible()

        row = [instance_name, fitness, distance, J1, J2, J3, J4, fleet_size, exec_time]
        data_alpha.append(row)

        # [BETA GA]
        folder = f'{opt_folder}betaGA_fleetsize_{num}/'
        fitness_df = pd.read_csv(folder+'optimization_iterations.csv')
        hyperparms_df = pd.read_csv(folder+'hyper-parameters.csv')
        costs_df = pd.read_csv(folder+'costs.csv')

        exec_time = float(hyperparms_df.loc[hyperparms_df.iloc[:, 0] == 'Algorithm time'].iloc[0, 1])
        fleet_size = hyperparms_df.loc[hyperparms_df.iloc[:, 0] == 'm'].iloc[0, 1]

        fitness = fitness_df.iloc[-1, 1]
        J1, w1 = costs_df.iloc[1,1], costs_df.iloc[0,1]
        J2, w2 = costs_df.iloc[1,2], costs_df.iloc[0,2]
        J3, w3 = costs_df.iloc[1,3], costs_df.iloc[0,3] 
        J4, w4 = costs_df.iloc[1,4], costs_df.iloc[0,4]
        J5, w5 = costs_df.iloc[1,5], costs_df.iloc[0,5]

        fleet = from_xml(folder+'result_instance.xml', True)
        fleet.iterate()
        feasible, distance, accept = fleet.feasible()

        row = [instance_name, fitness, distance, J1, J2, J3, J4, fleet_size, exec_time]
        data_beta.append(row)

In [5]:
alpha_df = pd.DataFrame(data_alpha, columns=columns)
#alpha_df.to_csv(f'../data/instances/alpha_results.csv')
alpha_df

,Instance,Fitness,Distance,J1,J2,J3,J4,Fleet size,Execution time (s)
0,c100cs15_20x20km,50652.34,719.31,1485.91,1283.82,185.56,1397304.52,11,3300.26
1,c50cs5_20x20km,9156.64,34.83,652.27,580.86,118.81,580094.15,5,455.78
2,c50cs8_20x20km,28146.79,590.05,782.66,669.58,117.15,917952.29,5,350.85
3,c75cs10_20x20km,13654.45,55.98,1089.29,931.25,174.18,889223.23,8,1380.16
4,c75cs15_20x20km,13903.79,66.57,1096.21,926.26,155.84,771198.36,9,1636.11


In [6]:
beta_df = pd.DataFrame(data_beta, columns=columns)
#beta_df.to_csv(f'../data/instances/beta_results.csv')
beta_df

,Instance,Fitness,Distance,J1,J2,J3,J4,Fleet size,Execution time (s)
0,c100cs15_20x20km,50618.23,684.12,1480.87,1278.39,186.31,1417261.98,11,1825.49
1,c50cs5_20x20km,9052.76,33.00,647.83,575.56,121.44,535234.95,5,304.94
2,c50cs8_20x20km,10465.24,220.08,781.18,667.72,139.44,889237.30,5,317.65
3,c75cs10_20x20km,13503.25,44.42,1078.07,923.44,163.22,838388.57,8,843.48
4,c75cs15_20x20km,13583.31,29.64,1080.20,913.59,170.91,646640.32,9,938.17


In [7]:
with pd.ExcelWriter('../data/instances/results.xlsx') as writer:
    alpha_df.to_excel(writer, sheet_name='alpha results', float_format="%.2f")
    beta_df.to_excel(writer, sheet_name='beta results', float_format="%.2f")